In [1]:
import numpy as np
import pandas as pd
import os
import ipython_bell
import random
import codecs

from astropy.io import fits
from astropy.table import Table

In [2]:
data = pd.read_csv('cutouts_Jack/y3_lens_search_systems_with_radii.csv', sep=',')#, usecols=[1,2,3,4,5,6])
print(data.columns)
data.head(1)

Index(['name', 'ra', 'dec', 'comment', 'rank', 'num_sources', 'num_lenses',
       'mem_match_id', 'einstein_radius', 'einstein_radius_err', 'ACT', 'BBNA',
       'BNA', 'TI', 'RNA', 'RNA2', 'redM', 'redM_missed', 'SPTPol', 'algo',
       'D17', 'J1', 'J2', 'aliases'],
      dtype='object')


,name,ra,dec,comment,rank,num_sources,num_lenses,mem_match_id,einstein_radius,einstein_radius_err,...,RNA,RNA2,redM,redM_missed,SPTPol,algo,D17,J1,J2,aliases
0,DESJ0002-3332,0.527213,-33.544062,"Not D17, not J1, not J2",4,2,2,-1,5.689696,0.828114,...,False,False,False,False,False,['BNA'],False,False,False,[]


In [3]:
data_log = pd.read_csv('cutouts_Jack/cutouts/table_service.log', sep='\t')#, usecols=[1,2,3,4,5,6])
print(data_log.columns)
data_log.head(1)

Index(['RA', 'DEC', 'COADD_OBJECT_ID', 'TILENAME', 'XSIZE', 'YSIZE',
       'MAKE_FITS', 'COLORS_FITS', 'MAKE_RGB_STIFF', 'RGB_STIFF_COLORS',
       'MAKE_RGB_LUPTON', 'RGB_LUPTON_COLORS', 'SEXAGECIMAL', 'RGB_MINIMUM',
       'RGB_STRETCH', 'RGB_ASINH'],
      dtype='object')


,RA,DEC,COADD_OBJECT_ID,TILENAME,XSIZE,YSIZE,MAKE_FITS,COLORS_FITS,MAKE_RGB_STIFF,RGB_STIFF_COLORS,MAKE_RGB_LUPTON,RGB_LUPTON_COLORS,SEXAGECIMAL,RGB_MINIMUM,RGB_STRETCH,RGB_ASINH
0,359.889732,2.139907,NaN,DES0000+0209,0.2,0.2,True,gri,False,gri,False,gri,DESJ235933.5357+020823.6652,1,50,10


In [4]:
x = np.zeros((1,3,46,46))
table_df = pd.DataFrame()
path_cut = '/Users/jimenagonzalez/research/DSPL/Searching-double-lenses/real_lenses/cutouts_Jack/cutouts/'
end_name = ['_g.fits', '_i.fits', '_r.fits']

for file in (os.listdir(path_cut)):
    if(file[-6:] == 'g.fits'):
        with fits.open(path_cut + file[:-7] + end_name[0]) as hdul: cutout_g = hdul[0].data
        with fits.open(path_cut + file[:-7] + end_name[1]) as hdul: cutout_i = hdul[0].data
        with fits.open(path_cut + file[:-7] + end_name[2]) as hdul: cutout_r = hdul[0].data
        data_ele = data_log[data_log['SEXAGECIMAL'] == file[:-7]]
        ra, dec = data_ele['RA'], data_ele['DEC']
        other_ele = data[data['ra'] == ra.values[0]]
        other_ele = other_ele[other_ele['dec'] == dec.values[0]]
        
        table_df = table_df.append(other_ele.iloc[0])#.str.encode(encoding='utf-8'))
        element = np.array([cutout_g, cutout_r, cutout_i])
        x = np.append(x, [element], axis = 0)
x = np.delete(x, 0, axis = 0)

In [5]:
print(x.shape)
print(len(table_df))

(249, 3, 46, 46)
249


In [6]:
my_types = {'name': np.dtype('U25'), 'ACT': 'int16', 'BBNA': 'int16','BNA': 'int16', 
            'TI': 'int16', 'RNA': 'int16', 'RNA2': 'int16', 'redM': 'int16', 'redM_missed': 'int16',
            'SPTPol': 'int16', 'D17': 'bool', 'J1': 'bool', 'J2': 'bool'}
table_df = table_df.astype(my_types)
table_df['name'] = table_df['name'].apply(lambda x: codecs.escape_decode(bytes(x.encode('unicode-escape').decode(), "utf-8"))[0].decode('utf-8').strip())
table_df['comment'] = table_df['comment'].apply(lambda x: codecs.escape_decode(bytes(x.encode('unicode-escape').decode(), "utf-8"))[0].decode('utf-8').strip())
#table_df['algo'] = table_df['algo'].apply(lambda x: codecs.escape_decode(bytes(x.encode('unicode-escape').decode(), "utf-8"))[0].decode('utf-8').strip())
#table_df['aliases'] = table_df['aliases'].apply(lambda x: codecs.escape_decode(bytes(x.encode('unicode-escape').decode(), "utf-8"))[0].decode('utf-8').strip())


/Users/jimenagonzalez/Software/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: invalid escape sequence '\u'
  


In [7]:
table_data = Table.from_pandas(table_df)
table_data[0:10]

name,ra,dec,comment,rank,num_sources,num_lenses,mem_match_id,einstein_radius,einstein_radius_err,ACT,BBNA,BNA,TI,RNA,RNA2,redM,redM_missed,SPTPol,algo,D17,J1,J2,aliases
str14,float64,float64,str108,float64,float64,float64,float64,float64,float64,int16,int16,int16,int16,int16,int16,int16,int16,int16,str39,bool,bool,bool,str73
DESJ0411-4506,62.792039,-45.10771,"yes D17, Not J1, not J2 skip, MMI 2094, substructure",3.0,1.0,1.0,2094.0,4.214248608926029,0.4967573989165634,0,0,0,0,1,0,1,0,0,"['RNA', 'redM']",True,False,False,[]
DESJ0218-3142,34.582248,-31.704951,"Not D17, Not J1, not J2 need to recheck fixed name, MMI 30, miscentered?",6.0,1.0,1.0,30.0,8.500516737236143,0.8898071970940211,1,0,0,0,1,0,0,1,1,"['RNA', 'ACT', 'SPTPol', 'redM_missed']",False,False,False,[]
DESJ0252-4732,43.08291,-47.54382,"Yes D17, not J1, Yes J2ad skip, MMI 12162",7.0,1.0,1.0,12162.0,2.9391850858525967,0.3944072637376033,0,0,0,0,0,0,1,0,0,['redM'],True,False,True,[]
DESJ0043-3453,10.98143,-34.887714,"Not D17, Not J1, not J2",7.0,1.0,1.0,-1.0,3.896357420091629,0.470090429014494,0,0,0,1,0,0,0,0,0,['TI'],False,False,False,[]
DESJ0047-3826,11.998543,-38.445888,"Not D17, Not J1, not J2, MMI 3088",7.0,2.0,1.0,3088.0,7.882081613104095,0.7166748244023192,1,0,0,1,0,0,0,1,0,"['TI', 'ACT', 'redM_missed']",False,False,False,[]
DESJ0557-4113,89.44746,-41.22415,"Yes D17, not J1, not J2 - skip",6.0,1.0,1.0,-1.0,15.766850479743244,1.5984694993980328,1,0,0,0,0,0,0,0,0,['ACT'],True,False,False,[]
DESJ0145-3541,26.44499,-35.690953,"Not D17, not J1, Yes J2n - skip, MMI 26429",10.0,1.0,1.0,26429.0,3.273814374167662,0.4199376210403969,0,0,0,0,0,0,1,0,0,['redM'],False,False,True,[]
DESJ2324-4944,351.13617,-49.74943,"Yes D17, not J1, not J2 skip, MMI 17188",7.0,2.0,1.0,17188.0,5.082352759796398,0.2876333749191082,0,0,0,0,0,0,1,0,0,['redM'],True,False,False,[]
DESJ0214-0535,33.533852,-5.592691,"Not D17, not J1, not J2, MMI 4168",7.0,2.0,2.0,4168.0,7.300699404957063,0.8232714665660213,0,0,0,0,0,0,1,0,0,['redM'],False,False,False,"['SL2S J021408-053532', 'SA22']"


In [8]:
name = 'real_lenses_Jack'
primary = fits.PrimaryHDU()
image = fits.ImageHDU(x, name="IMAGE")
table = fits.BinTableHDU(data = table_data)
hdu_list = fits.HDUList([primary, image, table])
hdu_list.writeto(path_cut + name + '.fits', overwrite=True)